In [28]:
import networkx as nx
import numpy as np
import community as comms
import pickle
from networkx.algorithms.community import greedy_modularity_communities
from networkx.generators.community import stochastic_block_model
from scipy.sparse import csr_matrix

In [43]:
def create_graph_and_node_mappings_from_file(filepath, dataset, node_mappings_dict, reverse_node_mappings_dict, graph_type='undirected'):
    if graph_type == 'undirected':
        G = nx.Graph()
        node_mappings = {}
        reverse_node_mappings = {}
    else:
        G = nx.DiGraph()
        node_mappings = node_mappings_dict[dataset]
        reverse_node_mappings = reverse_node_mappings_dict[dataset]
    with open(filepath, 'r') as f:
        counter = 0
        for line in f:
            edge = line.strip().split()
            node1 = str(edge[0])
            node2 = str(edge[1])
            if node1 not in node_mappings:
                node_mappings[node1] = counter
                reverse_node_mappings[counter] = node1
                counter += 1
            if node2 not in node_mappings:
                node_mappings[node2] = counter
                reverse_node_mappings[counter] = node2 
                counter += 1
            if graph_type == 'undirected' or graph_type == 'directed':
                source_node = node_mappings[node1]
                target_node = node_mappings[node2]
            elif graph_type == 'reverse_directed':
                source_node = node_mappings[node2]
                target_node = node_mappings[node1]
            G.add_edge(source_node, target_node)
    return G, node_mappings, reverse_node_mappings

def create_greedy_modularity_communities_dict(G, reverse_node_mappings):
    gm_communities = greedy_modularity_communities(G)
    node_communities_gm = {}
    community_counter = 0
    for community in gm_communities:
        for node in community:
            node_communities_gm[reverse_node_mappings[node]] = community_counter
        community_counter += 1
    return node_communities_gm, community_counter + 1

def create_louvain_communities_dict(G, reverse_node_mappings):
    node_communities_louvain = comms.best_partition(G)
    node_communities_louvain_original_ids = {}
    distinct_communities = []
    for reverse_node_id in node_communities_louvain:
        node_communities_louvain_original_ids[reverse_node_mappings[reverse_node_id]] = node_communities_louvain[reverse_node_id]
        if node_communities_louvain[reverse_node_id] not in distinct_communities:
            distinct_communities.append(node_communities_louvain[reverse_node_id])
    return node_communities_louvain_original_ids, len(distinct_communities)

def store_community_dict_as_file(community_dict, filepath):
    with open(filepath, 'wb') as handle:
        pickle.dump(community_dict, handle, protocol=2)

def calculate_edge_probabilities(G, communities, node_id_community_id_dict, reverse_node_mappings=None):
    between_community_stats = {}
    for edge in G.edges():
        # set source and target id
        source_id = edge[0]
        target_id = edge[1]
        if reverse_node_mappings is not None:
            source_id = reverse_node_mappings[source_id]
            target_id = reverse_node_mappings[target_id]
        source_community_id = node_id_community_id_dict[source_id]
        target_community_id = node_id_community_id_dict[target_id]
        if (source_community_id, target_community_id) not in between_community_stats:
            source_community_size = len(communities[source_community_id])
            target_community_size = len(communities[target_community_id])
            if G.is_directed():
                if source_community_id == target_community_id:
                    max_edge_count = (source_community_size*(source_community_size-1))
                    if has_selfloops(G):
                        max_edge_count += source_community_size
                else:
                    max_edge_count = 2*source_community_size*target_community_size
            else:
                if source_community_id == target_community_id:
                    max_edge_count = (source_community_size*(source_community_size-1))/2
                    if has_selfloops(G):
                        max_edge_count += source_community_size
                else:
                    max_edge_count = source_community_size*target_community_size                  
            between_community_stats[(source_community_id, target_community_id)] = {"existing_edge_count":0, "max_edge_count": max_edge_count}
            if not G.is_directed() and source_community_id != target_community_id:
                between_community_stats[(target_community_id, source_community_id)] = {"existing_edge_count":0, "max_edge_count": max_edge_count}
        between_community_stats[(source_community_id, target_community_id)]['existing_edge_count'] += 1
        if not G.is_directed() and source_community_id != target_community_id:
            between_community_stats[(target_community_id, source_community_id)]['existing_edge_count'] += 1
    for key in between_community_stats:
        between_community_stats[key]['edge_probability'] = between_community_stats[key]['existing_edge_count'] /  between_community_stats[key]['max_edge_count']
    rows = []
    cols = []
    data = []
    for key in between_community_stats:
        rows.append(key[0])
        cols.append(key[1])
        if between_community_stats[key]['edge_probability'] < 0 or between_community_stats[key]['edge_probability'] > 1:
            print(key)
        data.append(between_community_stats[key]['edge_probability'])
    communities_count = len(communities)
    return csr_matrix((data, (rows,cols)), shape=(communities_count, communities_count), dtype=float).todense().tolist()

def has_selfloops(G):
    return G.number_of_selfloops() > 0

def get_block_sizes(communities):
    block_sizes = []
    for community_id in communities:
        block_sizes.append(len(communities[community_id]))
    return block_sizes

def get_nodelist(communities, node_mappings=None):
    nodelist = []
    for community_id in communities:
        community = communities[community_id]
        for node_id in community:
            if node_mappings is not None:
                nodelist.append(node_mappings[node_id])
            else:
                nodelist.append(node_id)
    return nodelist

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)        

In [71]:
datasets = ['citeseer', 'cora', 'cora_full', 'PubMed']
graph_types = ['undirected', 'directed', 'reverse_directed']
community_count = {}
graphs = {}
node_mappings_dict = {}
reverse_node_mappings_dict = {}

data_directory = '../data/raw/'
communities_directory = '../data/community_id_dicts/'

In [72]:
for dataset in datasets:
    filepath = data_directory + dataset + '/' + dataset + '.cites'
    for graph_type in graph_types:
        G, node_mappings, reverse_node_mappings = create_graph_and_node_mappings_from_file(filepath, dataset, node_mappings_dict, reverse_node_mappings_dict, graph_type)
        graphs[dataset + '_' + graph_type] = G
        if graph_type == 'undirected':
            node_mappings_dict[dataset] = node_mappings
            reverse_node_mappings_dict[dataset] = reverse_node_mappings
            node_communities_gm, distinct_gm_count = create_greedy_modularity_communities_dict(G, reverse_node_mappings)
            node_communities_louvain, distinct_louvain_count = create_louvain_communities_dict(G, reverse_node_mappings)
            community_count[dataset] = {'distinct_gm_count': distinct_gm_count, 'distinct_louvain_count': distinct_louvain_count}
            store_community_dict_as_file(node_communities_gm, communities_directory + dataset + '/' + dataset + '_greedy_modularity.pickle')
            store_community_dict_as_file(node_communities_louvain, communities_directory + dataset + '/' + dataset + '_louvain.pickle')

In [73]:
data_directory = '../data/raw/'

In [74]:
# create_and_store_graph_transformations(data_directory + 'cora_full' + '/' + 'cora_full' + '.cites')
# create_and_store_graph_transformations_reverse_directed(data_directory + 'cora_full' + '/' + 'cora_full' + '.cites')

In [75]:
print(community_count)

{'citeseer': {'distinct_gm_count': 489, 'distinct_louvain_count': 468}, 'cora': {'distinct_gm_count': 106, 'distinct_louvain_count': 104}, 'cora_full': {'distinct_gm_count': 140, 'distinct_louvain_count': 45}, 'PubMed': {'distinct_gm_count': 115, 'distinct_louvain_count': 38}}


In [76]:
gm_node_id_to_community_id = {}
for dataset in datasets:
    with open(communities_directory + dataset + '/' + dataset + '_greedy_modularity.pickle', 'rb') as handle:
        gm_node_id_to_community_id[dataset] = pickle.load(handle)
        
louvain_node_id_to_community_id = {}
for dataset in datasets:
    with open(communities_directory + dataset + '/' + dataset + '_louvain.pickle', 'rb') as handle:
        louvain_node_id_to_community_id[dataset] = pickle.load(handle)
        

In [77]:
gm_community_id_to_node_id = {}

for dataset in datasets:
    gm_community_id_to_node_id[dataset] = {}
    for node_id in gm_node_id_to_community_id[dataset]:
        community_id = gm_node_id_to_community_id[dataset][node_id]
        if community_id not in gm_community_id_to_node_id[dataset]:
            gm_community_id_to_node_id[dataset][community_id] = []
        gm_community_id_to_node_id[dataset][community_id].append(node_id)


In [78]:
louvain_community_id_to_node_id = {}

for dataset in datasets:
    louvain_community_id_to_node_id[dataset] = {}
    for node_id in louvain_node_id_to_community_id[dataset]:
        community_id = louvain_node_id_to_community_id[dataset][node_id]
        if community_id not in louvain_community_id_to_node_id[dataset]:
            louvain_community_id_to_node_id[dataset][community_id] = []
        louvain_community_id_to_node_id[dataset][community_id].append(node_id)

In [79]:
gm_edge_probabilities = {}
gm_block_sizes = {}
gm_nodelists = {}

louvain_edge_probabilities = {}
louvain_block_sizes = {}
louvain_nodelists = {}

for dataset in datasets:
    for graph_type in graph_types:
        G = graphs[dataset + '_' + graph_type]
        node_mappings = node_mappings_dict[dataset]
        reverse_node_mappings = reverse_node_mappings_dict[dataset]
        gm_edge_probabilities[dataset + '_' + graph_type] = calculate_edge_probabilities(G, gm_community_id_to_node_id[dataset], gm_node_id_to_community_id[dataset], reverse_node_mappings)
        gm_block_sizes[dataset + '_' + graph_type] = get_block_sizes(gm_community_id_to_node_id[dataset])
        gm_nodelists[dataset + '_' + graph_type] = get_nodelist(gm_community_id_to_node_id[dataset], node_mappings_dict[dataset])
        louvain_edge_probabilities[dataset + '_' + graph_type] = calculate_edge_probabilities(G, louvain_community_id_to_node_id[dataset], louvain_node_id_to_community_id[dataset], reverse_node_mappings)
        louvain_block_sizes[dataset + '_' + graph_type] = get_block_sizes(louvain_community_id_to_node_id[dataset])
        louvain_nodelists[dataset + '_' + graph_type] = get_nodelist(louvain_community_id_to_node_id[dataset], node_mappings_dict[dataset])

In [80]:
gm_stochastic_block_model_graphs = {}
louvain_stochastic_block_model_graphs = {}

In [81]:
# create greedy modularity-based stochastic block model graphs
for dataset in datasets:
    for graph_type in graph_types:
        gm_stochastic_block_model_graphs[dataset + '_' + graph_type] =  []
        louvain_stochastic_block_model_graphs[dataset + '_' + graph_type] =  []
        original_graph = graphs[dataset + '_' + graph_type]
        print(original_graph.is_directed())
        for seed in range(10):
            G = stochastic_block_model(gm_block_sizes[dataset + '_' + graph_type],
                                       gm_edge_probabilities[dataset + '_' + graph_type],
                                       gm_nodelists[dataset + '_' + graph_type],
                                       seed,
                                       original_graph.is_directed(),
                                       has_selfloops(graphs[dataset + '_' + graph_type]),
                                       True)
            gm_stochastic_block_model_graphs[dataset + '_' + graph_type].append({'seed':seed, 'graph':G})
            G = stochastic_block_model(louvain_block_sizes[dataset + '_' + graph_type],
                                       louvain_edge_probabilities[dataset + '_' + graph_type],
                                       louvain_nodelists[dataset + '_' + graph_type],
                                       seed,
                                       original_graph.is_directed(),
                                       has_selfloops(graphs[dataset + '_' + graph_type]),
                                       True)
            louvain_stochastic_block_model_graphs[dataset + '_' + graph_type].append({'seed':seed, 'graph':G})


False
True
True
False
True
True
False
True
True
False
True
True


In [82]:
for dataset in datasets:
    for graph_type in graph_types:
        print(dataset + '_' + graph_type)
        for entry in gm_stochastic_block_model_graphs[dataset + '_' + graph_type]:
            print(len(entry['graph'].edges()))
        print()

citeseer_undirected
4735
4724
4780
4736
4610
4731
4703
4667
4640
4682

citeseer_directed
4557
4601
4560
4674
4667
4580
4609
4684
4613
4666

citeseer_reverse_directed
4537
4576
4581
4680
4676
4577
4599
4700
4624
4643

cora_undirected
5304
5209
5290
5311
5346
5321
5231
5256
5118
5337

cora_directed
5207
5105
5187
5051
5127
5321
5156
5177
5042
5193

cora_reverse_directed
5189
5113
5167
5105
5145
5329
5146
5164
5056
5157

cora_full_undirected
62653
62340
62152
62438
62414
62008
62406
62524
62303
62013

cora_full_directed
61750
61650
61533
61726
61537
61733
61399
61665
61552
61030

cora_full_reverse_directed
61711
61636
61533
61743
61486
61645
61365
61709
61504
60990

PubMed_undirected
44588
44438
44115
44268
44533
44113
44440
44504
44186
44230

PubMed_directed
42257
42290
41998
42325
41935
42201
42088
42144
42179
41865

PubMed_reverse_directed
42259
42291
41984
42318
41955
42176
42097
42114
42145
41915



In [87]:
for dataset in datasets:
    for graph_type in graph_types:
        print(dataset + '_' + graph_type)
        for entry in louvain_stochastic_block_model_graphs[dataset + '_' + graph_type]:
            print(len(entry['graph'].edges()))
        print()

citeseer_undirected
4669
4683
4702
4654
4731
4650
4635
4676
4630
4778

citeseer_directed
4593
4566
4583
4657
4577
4717
4485
4562
4494
4625

citeseer_reverse_directed
4586
4579
4570
4622
4569
4682
4480
4567
4470
4628

cora_undirected
5338
5257
5219
5317
5322
5333
5268
5270
5285
5366

cora_directed
5198
5167
5063
5059
5113
5180
5067
5184
5055
5229

cora_reverse_directed
5203
5158
5133
5080
5113
5162
5011
5177
5056
5233

cora_full_undirected
62695
62180
62159
61909
62697
62075
62509
62772
62079
62341

cora_full_directed
60442
59932
60037
60160
60088
60279
59605
60242
60171
60228

cora_full_reverse_directed
60499
59979
60059
60206
60095
60313
59626
60249
60169
60244

PubMed_undirected
44674
44152
44414
44353
44251
44065
44364
44523
44210
44076

PubMed_directed
41198
41047
41045
40872
40802
40888
41114
40921
40809
40791

PubMed_reverse_directed
41215
40991
41000
40911
40805
40836
41077
40934
40773
40735



In [88]:
for dataset in graphs:
    print(dataset)    
    print(len(graphs[dataset].edges()))

citeseer_undirected
4676
citeseer_directed
4732
citeseer_reverse_directed
4732
cora_undirected
5278
cora_directed
5429
cora_reverse_directed
5429
cora_full_undirected
62421
cora_full_directed
64259
cora_full_reverse_directed
64259
PubMed_undirected
44327
PubMed_directed
44338
PubMed_reverse_directed
44338


In [89]:
for dataset in datasets:
    for graph_type in graph_types:
        for entry in gm_stochastic_block_model_graphs[dataset + '_' + graph_type]:
            with open('../data/sbm_graphs/greedy_modularity_based/' + dataset + '/' + dataset + '_' + graph_type + '_sbm_gm_seed_' + str(entry['seed']) + '.cites', 'a') as output_file:
                for edge in list(entry['graph'].edges()):
                    source_id = reverse_node_mappings_dict[dataset][edge[0]]
                    target_id = reverse_node_mappings_dict[dataset][edge[1]]
                    output_file.write(source_id + ' ' + target_id + '\n')

In [90]:
for dataset in datasets:
    for graph_type in graph_types:
        for entry in louvain_stochastic_block_model_graphs[dataset + '_' + graph_type]:
            with open('../data/sbm_graphs/louvain_based/' + dataset + '/' + dataset + '_' + graph_type + '_sbm_louvain_seed_' + str(entry['seed']) + '.cites', 'a') as output_file:
                for edge in list(entry['graph'].edges()):
                    source_id = reverse_node_mappings_dict[dataset][edge[0]]
                    target_id = reverse_node_mappings_dict[dataset][edge[1]]
                    output_file.write(source_id + ' ' + target_id + '\n')